In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# libraries
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import numpy as np
import time
import itertools
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingRegressor

import random
random.seed(7)
tf.random.set_seed(7)

In [ ]:
directory = '../input/ventilator-pressure-prediction'
train = pd.read_csv(os.path.join(directory, 'train.csv'))
test = pd.read_csv(os.path.join(directory, 'test.csv'))
sub = pd.read_csv(os.path.join(directory, 'sample_submission.csv'))

In [ ]:
train[['R', 'C', 'time_step', 'u_in', 'u_out']].values

In [ ]:
X = train[['R', 'C', 'time_step', 'u_in', 'u_out']].values
Y = train.pressure.values

sample_length = 80
input_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
  X, None, sequence_length=sample_length, sequence_stride=sample_length)
target_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
  Y, None, sequence_length=sample_length, sequence_stride=sample_length)

for batch in zip(input_dataset, target_dataset):
  inputs, targets = batch
  assert np.array_equal(inputs[0], X[:sample_length])

  # second sample equals output timestamps 20-40
  assert np.array_equal(targets[1], Y[sample_length:2*sample_length])
  break

In [ ]:
import time
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingRegressor

In [ ]:
train.shape

In [ ]:
def lstm_model():
    # create model
    model = Sequential()
    model.add(Input(shape = (inputs.shape[1], inputs.shape[2])))
    model.add(LSTM(320, return_sequences=True))
    model.add(LSTM(320, return_sequences=True))
    model.add(Dense(160, activation = 'relu'))
    #model.add(Dense(80, activation = 'relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mae', optimizer='adam', metrics = [tensorflow.keras.metrics.RootMeanSquaredError()])
    return model

In [ ]:
def bi_lstm_model():
    # create model
    model = Sequential()
    model.add(Input(shape = (inputs.shape[1], inputs.shape[2])))
    model.add(Bidirectional(LSTM(640, return_sequences=True)))
    model.add(Bidirectional(LSTM(320, return_sequences=True)))
    model.add(LSTM(320, return_sequences=True))
    model.add(LSTM(320, return_sequences=True))
    model.add(Dense(320, activation = 'relu'))
    model.add(Dense(160, activation = 'relu'))
    #model.add(Dense(80, activation = 'relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mae', optimizer='adam', metrics = [tensorflow.keras.metrics.RootMeanSquaredError()])
    return model

In [ ]:
def cnn_lstm_model():
    # create model

    model = Sequential()
    model.add(Input(shape = ( inputs.shape[1], inputs.shape[2])))
    model.add(Conv1D(filters = 320, kernel_size = 3, strides = 1, padding = 'causal', activation = "relu" ))
    model.add(LSTM(320, return_sequences=True, activation = 'tanh'))
    model.add(LSTM(320, return_sequences=True, activation = 'tanh'))
    model.add(Dense(160, activation = 'relu'))
    #model.add(Dense(80, activation = 'relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mae', optimizer = tensorflow.keras.optimizers.Adam(), metrics = [tensorflow.keras.metrics.RootMeanSquaredError()])
    return model

In [ ]:
# define the stages of the pipeline
pipeline = Pipeline(steps= [('model', KerasRegressor(build_fn=cnn_lstm_model, epochs=500, batch_size = 512, verbose=1))])

# fit the pipeline model with the training data                            
pipeline.fit(inputs, targets)

In [ ]:
4024000/80

In [ ]:
sub_array = pipeline.predict(test[['R', 'C', 'time_step', 'u_in', 'u_out']].to_numpy().reshape(50300, 80, 5))
#list(itertools.chain(*sub_array))

In [ ]:
sub_array =list(itertools.chain(*sub_array))

In [ ]:
#sub_array = pipeline.predict(test[['R', 'C', 'time_step', 'u_in', 'u_out']])

In [ ]:
pd.DataFrame({'id':test.id, 'pressure':sub_array}).to_csv('mark_5.csv', index=False)

Building from the top down: Implementation of deep-lstms